In [1]:
import sys
sys.path.append("..")

In [2]:
from src.datasets.hateexplainrace import HateXplainRace
from src.demonstrations.random import RandomSampler

In [3]:
from src.models.gpt import GPT
from src.models.chatgpt import ChatGPT
from src.models.hf import HF

In [4]:
from src.utils import metrics

In [5]:
def build_model(model_name: str, model_params):
    """Builds model class from model name and params provided

    :param model_name: name of model being used
    :type model_name: str
    :param model_params: list of parameters provided for each model
    :type model_params: Dict[str, Any]
    :raises ValueError: model does not exist
    :return: fully formed model
    :rtype: APIModel
    """    
    models = {
        "gpt3": GPT("text-curie-001", **model_params),
        "chatgpt": ChatGPT("gpt-3.5-turbo", **model_params),
        "flan": HF("https://api-inference.huggingface.co/models/google/flan-t5-large", **model_params),
        "ul2": HF("https://api-inference.huggingface.co/models/google/ul2", **model_params),
    }

    model = None

    try:
        model = models[model_name]
    except KeyError:
        raise ValueError(f"{model_name} does not exist!")

    return model

In [6]:
import random

In [13]:
hate = HateXplainRace('data/HateXplain')

In [14]:
train_prompts, test_prompts, test_labels, test_demographics = hate.create_prompts()

In [15]:
fiveshotSampler = RandomSampler(shots=5)

In [16]:
fiveshot_prompts = fiveshotSampler.create_demonstrations(train_prompts, test_prompts)

100%|██████████| 1924/1924 [00:00<00:00, 259605.63it/s]


In [17]:
zeroshotSampler = RandomSampler(shots=0)

In [18]:
zeroshot_prompts = zeroshotSampler.create_demonstrations(train_prompts, test_prompts)

100%|██████████| 1924/1924 [00:00<00:00, 1310677.42it/s]


In [19]:
fiveshot_prompts_sample, fiveshot_labels, five_shot_demographics = zip(*random.sample(list(zip(fiveshot_prompts, test_labels, test_demographics)), 100))

In [20]:
zeroshot_prompts_sample, zeroshot_labels, zeroshot_demographics = zip(*random.sample(list(zip(zeroshot_prompts, test_labels, test_demographics)), 100))

In [21]:
scores = {
    'chatgpt' :{
        'fiveshot' : {
            'race' : [],
            'gender' : []
        },
        'zeroshot': {
             'race' : [],
            'gender' : []
        }
    },
    'gpt3' : {
        'fiveshot' : {
            'race' : [],
            'gender' : []
        },
        'zeroshot': {
             'race' : [],
            'gender' : []
        }
    }
}

for i in range(5):
    for model_name in ["chatgpt", "gpt3"]:
        model = build_model(model_name, {"temperature" : 1, "max_tokens" : 5})

        fiveshot_responses = model.generate_from_prompts(fiveshot_prompts_sample)

        five_shot_score_race = metrics(fiveshot_responses, fiveshot_labels, "hatexplain-race", five_shot_demographics)['total_score']
        five_shot_score_gender = metrics(fiveshot_responses, fiveshot_labels, "hatexplain-gender", five_shot_demographics)['total_score']

        scores[model_name]['fiveshot']['race'].append(five_shot_score_race)
        scores[model_name]['fiveshot']['gender'].append(five_shot_score_gender)

        zeroshot_responses = model.generate_from_prompts(zeroshot_prompts_sample)

        zero_shot_score_race = metrics(zeroshot_responses, zeroshot_labels, "hatexplain-race", zeroshot_demographics)['total_score']
        zero_shot_score_gender = metrics(zeroshot_responses, zeroshot_labels, "hatexplain-gender", zeroshot_demographics)['total_score']

        scores[model_name]['zeroshot']['race'].append(zero_shot_score_race)
        scores[model_name]['zeroshot']['gender'].append(zero_shot_score_gender)

100%|██████████| 100/100 [00:45<00:00,  2.21it/s]
/home/ksasse/miniconda3/envs/fairness/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/home/ksasse/code/fairness/src/utils.py:120: RuntimeWarning: invalid value encountered in divide
  recall = tp / (tp + fn)
/home/ksasse/code/fairness/src/utils.py:121: RuntimeWarning: invalid value encountered in divide
  specificity = tn / (tn + fp)
/home/ksasse/miniconda3/envs/fairness/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))